# Sentiment Analysis

This notebook intends to build a sentiment classifier (positive, negative) from “Multi-Domain Sentiment Dataset” per each category (“Books”, “DVD”, “Electronics”, “Kitchen”).

### Library imports

In [1]:
#Scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

#Libraries to graph
import matplotlib.pyplot as plt
import seaborn as sns

#NLTK
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd


stemmer = nltk.stem.SnowballStemmer('english') 
nltk.download('stopwords') 

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


False

### Read and transform the .review's files

Run the python file "PreProcessingSentimentAnalysis.py"

### Creating the training/validation dataframe

In [4]:
def create_df (file_name):
    df = pd.read_csv(file_name, sep=',')


                                                 review     label
83     title title recommend it though a looking you...  positive
914    reading this i've elements etc jian i <dash-n...  positive
808    t anyone a fascinating them in storyline and ...  positive
1087   i i i i i she says old woman &quot;better nec...  positive
1363   much more full of parrot guides make more rea...  positive


### Text processing function

In [ ]:
def text_processing(text):
    # Step 1: Remove special characters using a regular expression (non-words).
    processed_feature = re.sub(r'\W', ' ', str(text))
    # Step 2: Remove single-character occurrences.
    processed_feature = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    # Step 3: Remove numbers (very sporadic occurrences in our dataset).
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature)
    # Step 4: Simplify consecutive spaces to a single space between words.
    processed_feature = re.sub(' +', ' ', processed_feature)
    # Step 5: Convert all text to lowercase.
    processed_feature = processed_feature.lower()
    # Step 6: Apply stemming. It's a way to bring words to a common root, simplifying the vocabulary.
    # This helps to avoid having two different words with the same meaning in our vocabulary.
    processed_feature = " ".join([stemmer.stem(i) for i in processed_feature.split()])

    return processed_feature
